In [1]:
########################################################################################################################
# This script visualizes the decision curve analysis for longitudinal prediction models
# Remark: This script adopts the visual configuration of a 3x3 panels. Please revise it accordingly if you adopt a 
# different setting. 
########################################################################################################################

In [ ]:
########################################################################################################################
# Import packages
########################################################################################################################
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["font.sans-serif"] = ["Century Gothic"]
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
########################################################################################################################
# USER_SPECIFIC SETTING
# C_LIST: A list of different numbers of feature encounteres to be included
# D_LIST: A list of different maximum widths of the look-back window in days
# ABLATION: Boolean. False for pre-ablation modeling and True for post-ablation modeling
# IMPUTE: A string representing the imputation method adopted (in 'Zero', 'Mean', and 'Median')
# CSL: Boolean. False for standard learning and True for cost-sensitive learning
# IN_DIR: The input path of the directories storing the modeling results 
# (e.g., the directory with the subdirectory RiskPath_Results/) 
# OUT_DIR: The output path of the directory storing the overall performance statistics
########################################################################################################################
C_LIST: list[int] = [2, 3, 4]
D_LIST: list[int] = [60, 120, 180]
ABLATION: bool = False
IMPUTE: str = 'Zero'
CSL: bool = False
IN_DIR: str = ''
OUT_DIR: str = '_Final_Results/'

In [ ]:
########################################################################################################################
# Specify the directories to extract probability estimates
########################################################################################################################
ablate_str: str = '_Ablated' if ABLATION else ''
csl_str: str = '_CSL' if CSL else ''
files_needed: dict[tuple[int, int], str] = {}
for C, D in product(C_LIST, D_LIST):
    dir_path: str = os.path.join(IN_DIR, f'RiskPath_Results{ablate_str}/Predicted_Probabilities/')
    file_path: str = dir_path + f'{C}_encounters_{D}_days_{IMPUTE}{ablate_str}{csl_str}.csv'    
    files_needed[(C, D)] = file_path
for k, v in files_needed.items():
    assert os.path.exists(v)

In [5]:
########################################################################################################################
# Define a function to compute standardized net benefit (sNB)
########################################################################################################################
def nb(y_true_, y_prob_, pt, standard=True):
    y_pred_ = (y_prob_ >= pt).astype(int)
    TP = np.sum((y_pred_ == 1) & (y_true_ == 1))
    FP = np.sum((y_pred_ == 1) & (y_true_ == 0))
    n = len(y_true_)
    nb = (TP / n) - (FP / n) * (pt / (1 - pt))
    snb = nb / np.mean(y_true_)
    return snb if standard else nb

In [6]:
########################################################################################################################
# Define other relevant functions to compute sNB (from counts and treat all)
########################################################################################################################
def nb_from_counts(tp, fp, n, pt, prevalence, standard=True):
    nb = (tp / n) - (fp / n) * (pt / (1 - pt))
    return nb / prevalence if standard else nb

def compute_treat_all_nb(y_true, pts, standard=True):
    n = len(y_true)
    prev = np.mean(y_true)
    tp = np.sum(y_true == 1)
    fp = np.sum(y_true == 0)
    return [nb_from_counts(tp, fp, n, pt, prev, standard=standard) for pt in pts]


In [7]:
########################################################################################################################
# Create dictionaries to save true labels, probabilities, and sNB values
########################################################################################################################
true_dict: dict = {}
prob_dict: dict = {}
nb_dict: dict = {}

In [35]:
########################################################################################################################
# Define the policy thresholds for the computation of sNB
########################################################################################################################
POLICY_PROBS = np.array([i / 1000 for i in range(1, 1000)], dtype=float)
assert np.all((POLICY_PROBS > 0) & (POLICY_PROBS < 1)), 'Each element in POLICY_PROBS must be strictly within (0, 1)'

In [37]:
########################################################################################################################
# Load the probability estimates files
########################################################################################################################
for config, file_path in files_needed.items():
    df_cur: pd.DataFrame = pd.read_csv(file_path)

    ####################################################################################################################
    # Extract the true labels and estimated probabilities
    # Compute and store the standardized net benefit (sNB) for each decision threshold in POLICY_PROBS
    ####################################################################################################################
    y_test = df_cur['y_test'].to_numpy()
    y_prob = df_cur['y_prob'].to_numpy()

    true_dict[config] = y_test
    prob_dict[config] = y_prob
    nb_dict[config] = [nb(y_test, y_prob, pt, standard=True) for pt in POLICY_PROBS]

In [ ]:
########################################################################################################################
# Create the 3x3 subplot figure
########################################################################################################################
configs = sorted(files_needed.keys())
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 7))
axes = axes.ravel()
for i, config in enumerate(configs):
    ax = axes[i]
    y_true = true_dict[config]
    nb_vals = np.asarray(nb_dict[config], dtype=float)
    ax.plot(POLICY_PROBS, nb_vals, label=f'RiskPath', linewidth=3, color='tab:blue')
    ax.plot(POLICY_PROBS, compute_treat_all_nb(y_true, POLICY_PROBS, standard=True), linestyle='--', label='Treat all', color='tab:brown', linewidth=3)
    ax.axhline(0.0, linestyle='-', linewidth=3, label='Treat none', color='black')
    ax.grid(alpha=0.5)
    ax.set_ylim(-0.1, 1.05)
    ax.set_xlim(0, 0.8)      # Revise if needed
    ax.tick_params(axis='both', which='major', labelsize=13)

for j in range(len(configs), len(axes)):
    axes[j].axis('off')

col_labels = D_LIST
for col, d in enumerate(col_labels):
    fig.text(0.2 + col * 0.32, 0.93, f'{d} lookback days',
             ha='center', va='bottom', fontsize=20, fontweight='bold')
row_labels = C_LIST
for row, c in enumerate(row_labels):
    fig.text(-0.02, 0.80 - row * 0.29, f'{c} encounters',
             ha='left', va='center', fontsize=20, fontweight='bold', rotation=90)

fig.supxlabel('Threshold probability (p' + r'$_t$' +')', fontsize=20)
fig.supylabel('Standardized net benefit (sNB)', fontsize=20, x=0.01)
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, 
           bbox_to_anchor=(0.5, 1.07), ncols=4, loc='upper center', frameon=True, 
           fontsize=20)         
plt.tight_layout(rect=[0, 0, 1, 0.94])
out_path = os.path.join(OUT_DIR, f"DCA_Longitudinal_sNB_{IMPUTE}{ablate_str}{csl_str}.png")
plt.savefig(out_path, dpi=300, bbox_inches="tight")
plt.show()